In [1]:
import os
import json
import logging
import sys
import warnings
from typing import Dict, Any

# --- 1. GLOBAL SILENCING CONFIGURATION ---
os.environ["TQDM_DISABLE"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
warnings.filterwarnings("ignore")

# --- 2. LOGGING SETUP ---
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

noisy_loggers = ["sentence_transformers", "transformers", "urllib3", "requests", "huggingface_hub", "filelock", "tqdm"]
for logger_name in noisy_loggers:
    logging.getLogger(logger_name).setLevel(logging.ERROR)

from builder import CausalGraphBuilder
from bunny_retriever import BunnyPathRetriever
from explainer import CausalGraphExplainer

In [2]:
import logging
from typing import List, Tuple, Dict, Any
from builder import CausalGraphBuilder
from bunny_retriever import BunnyPathRetriever

# Setup logging to see the retrieval process
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BunnyRAGChain:
    def __init__(self, model_name: str = "all-mpnet-base-v2", graph_path: str = "causal_math_graph_llm.json"):
        # Added encoding="utf-8" safety within the builder if it has a load method
        self.builder = CausalGraphBuilder(model_name=model_name)
        self.builder.load(graph_path) 
        self.retriever = BunnyPathRetriever(self.builder)
        self.graph_path = graph_path

    def explore_and_query(self, query: str):
        # 1. Retrieve ranked nodes using Effective Resistance
        top_results = self.retriever.retrieve_nodes_part2(
            query=query, 
            top_k=15, 
            labda=0.02, 
            json_path=self.graph_path
        )

        if not top_results:
            return "No direct causal paths found in the knowledge graph."

        # 2. Convert Node IDs to human-readable text
        # This is vital so the evaluator sees 'triangle' instead of 'node_123'
        retrieved_texts = [self.builder.node_text.get(nid, nid) for nid, score in top_results]

        # 3. Format context as a comma-separated string for the evaluator
        return ", ".join(retrieved_texts)

# --- Execution ---
if __name__ == "__main__":
    # Ensure causal_math_graph_llm.json is in your directory
    chain = BunnyRAGChain()
    
    # Example Query
    nodes = chain.explore_and_query('Tell me about surface tension and minimal surfaces.')

[('surface tension', 0.7634661197662354), ('repulsion between film surfaces', 0.45514827966690063), ('minimal area', 0.4434366226196289), ('discovery of a new surface', 0.3672070801258087), ('inhomogeneous surface concentrations', 0.36407333612442017)]


In [3]:
import logging
from typing import List, Tuple, Dict, Any
from builder import CausalGraphBuilder
from bunny_retriever import BunnyPathRetriever

class BunnyRAGChain:
    def __init__(self, model_name: str = "all-mpnet-base-v2", graph_path: str = "causal_math_graph_llm.json"):
        # Load the graph and initialize components
        self.builder = CausalGraphBuilder(model_name=model_name)
        self.builder.load(graph_path)
        self.retriever = BunnyPathRetriever(self.builder)
        self.graph_path = graph_path

    def generate_answer(self, query: str, retrieved_nodes: List[str]) -> str:
        """
        Synthesizes a sentence from the retrieved causal concepts.
        In a full RAG pipeline, this would be passed to an LLM.
        """
        if not retrieved_nodes:
            return "I couldn't find any causal relationships in the knowledge base to answer that."
        
        # Take the top 3-4 most relevant concepts
        concepts = ", ".join(retrieved_nodes[:4])
        
        # Simple template-based generation for demonstration
        return f"Regarding '{query}', the causal graph indicates relevance to: {concepts}."

    def explore_and_query(self, query: str):
        # 1. Retrieve ranked nodes using Effective Resistance
        top_results = self.retriever.retrieve_nodes_part2(
            query=query, 
            top_k=5, 
            labda=-0.01, 
            json_path=self.graph_path
        )

        if not top_results:
            return "No direct causal paths found."

        # 2. Map Node IDs to text
        retrieved_texts = [self.builder.node_text.get(nid, nid) for nid, score in top_results]

        # 3. Generate the final sentence
        return self.generate_answer(query, retrieved_texts)

In [4]:
# --- Execution with Test Questions ---
if __name__ == "__main__":
    chain = BunnyRAGChain()
    
    test_questions = [
        'What happens when the circumcenter is on the side of the triangle?',
        "how many side of squares",
        "what is the circumference of circle",
        "How is the area of a square related to its side length?",
        "What conditions make two triangles similar?",
        "What characterizes a tangential quadrilateral?",
        "What happens to a graph when a cut-set is removed?",
        "How is Brownian motion related to the diffusion constant?",
        "Why do soap films form minimal area surfaces?",
        "What is the relationship between snarks and the four-color theorem?"
    ]

    # print(f"{'Question':<70} | {'Generated Causal Explanation'}")
    # print("-" * 130)
    
    for question in test_questions:
        answer = chain.explore_and_query(question)
        print(f"{answer}")
        print()

[('a triangle changes shape', 0.5867527723312378), ('triangles are similar', 0.5511089563369751), ('a triangle will not change shape', 0.5075494050979614), ('tessellated triangles', 0.4814336895942688), ("the 'circles' in taxicab geometry", 0.4810835123062134)]
Regarding 'What happens when the circumcenter is on the side of the triangle?', the causal graph indicates relevance to: minor-closed property, squared integer, inhomogeneous surface concentrations, convincing evidence of Brownian motion.

[('construction of a square with a given side', 0.6614404916763306), ('squares tilted at 45° to the coordinate axes', 0.5511420965194702), ('a square is a regular polygon', 0.5356656908988953), ('formula for the area of a square as the second power of its side length', 0.5123594999313354), ('specifying the lengths of all three sides', 0.5033279657363892)]
Regarding 'how many side of squares', the causal graph indicates relevance to: minor-closed property, squared integer, inhomogeneous surface